In [1]:
import torch
from yolov6.backbone import *
from yolov6.neck import *

In [2]:
b = EfficientRep()

In [3]:
rand_in = torch.rand((1, 3, 416, 415))

In [4]:
x = b(rand_in)

In [5]:
len(x)

4

In [6]:
x[0].shape, x[1].shape, x[2].shape,  x[3].shape

(torch.Size([1, 64, 104, 104]),
 torch.Size([1, 128, 52, 52]),
 torch.Size([1, 256, 26, 26]),
 torch.Size([1, 512, 13, 13]))

In [7]:
n = RepBiPAN()

In [8]:
o = n(x)

torch.Size([1, 128, 13, 13])


In [9]:
o[0].shape, o[1].shape, o[2].shape

(torch.Size([1, 64, 52, 52]),
 torch.Size([1, 128, 26, 26]),
 torch.Size([1, 256, 13, 13]))

In [10]:
from yolov6.head import *

In [11]:
h = DetectionHead()

In [12]:
o = h(o)

In [13]:
o[0].shape, o[1].shape, o[2].shape

(torch.Size([1, 13, 13, 25]),
 torch.Size([1, 26, 26, 25]),
 torch.Size([1, 52, 52, 25]))

In [14]:
from yolov6.yolov6 import YOLOv6s

model = YOLOv6s()

image = torch.rand(1, 3, 416, 416)

In [15]:
out = model(image)

torch.Size([1, 128, 13, 13])


In [16]:
out[0].shape, out[1].shape, out[2].shape

(torch.Size([1, 13, 13, 25]),
 torch.Size([1, 26, 26, 25]),
 torch.Size([1, 52, 52, 25]))

In [17]:
from utils.to_onnx import to_onnx

In [18]:
from torchinfo import summary
summary(model, (1, 3, 416, 416))

torch.Size([1, 128, 13, 13])


Layer (type:depth-idx)                                       Output Shape              Param #
YOLOv6s                                                      [1, 13, 13, 25]           --
├─EfficientRep: 1-1                                          [1, 64, 104, 104]         --
│    └─RepConv: 2-1                                          [1, 32, 416, 416]         --
│    │    └─Conv2d: 3-1                                      [1, 32, 416, 416]         896
│    │    └─BatchNorm2d: 3-2                                 [1, 32, 416, 416]         64
│    │    └─ReLU: 3-3                                        [1, 32, 416, 416]         --
│    └─ModuleList: 2-2                                       --                        --
│    │    └─Sequential: 3-4                                  [1, 32, 208, 208]         19,808
│    │    └─Sequential: 3-5                                  [1, 64, 104, 104]         101,568
│    │    └─Sequential: 3-6                                  [1, 128, 52, 52]        

In [19]:
to_onnx(model, image, 'model.onnx')

torch.Size([1, 128, 13, 13])
Model has been successfully converted to ONNX format and saved to model.onnx


In [20]:
from ptflops import get_model_complexity_info

flops, params = get_model_complexity_info(model, (3, 416, 416), as_strings=True, print_per_layer_stat=True)
print(f'FLOPs: {flops}')
print(f'Parameters: {params}')


torch.Size([1, 128, 13, 13])
YOLOv6s(
  17.88 M, 100.000% Params, 10.99 GMac, 99.798% MACs, 
  (backbone): EfficientRep(
    11.53 M, 64.488% Params, 8.08 GMac, 73.348% MACs, 
    (repconv1): RepConv(
      960, 0.005% Params, 171.67 MMac, 1.559% MACs, 
      (conv): Conv2d(896, 0.005% Params, 155.06 MMac, 1.408% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, 0.000% Params, 11.08 MMac, 0.101% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(0, 0.000% Params, 5.54 MMac, 0.050% MACs, inplace=True)
    )
    (layers): ModuleList(
      (0): Sequential(
        19.81 k, 0.111% Params, 861.13 MMac, 7.819% MACs, 
        (0): RepConv(
          9.31 k, 0.052% Params, 404.26 MMac, 3.671% MACs, 
          (conv): Conv2d(9.25 k, 0.052% Params, 400.11 MMac, 3.633% MACs, 32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (bn): BatchNorm2d(64, 0.000% Params, 2.77 MMac, 0.025% MACs, 32, eps=1e-